In [7]:
import pandas as pd
import numpy as np
import torch

dataset: https://www.kaggle.com/datasets/alfrandom/protein-secondary-structure

In [2]:
path = "extra_data/2018-06-06-pdb-intersect-pisces.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa,Exptl.,resolution,R-factor,FreeRvalue
0,1FV1,F,NPVVHFFKNIVTPRTPPPSQ,CCCCCBCCCCCCCCCCCCCC,CCCCCECCCCCCCCCCCCCC,20,False,XRAY,1.90,0.23,0.27
1,1LM8,H,DLDLEMLAPYIPMDDDFQLR,CCCCCCCCCBCCSCCCEECC,CCCCCCCCCECCCCCCEECC,20,False,XRAY,1.85,0.20,0.24
2,1O06,A,EEDPDLKAAIQESLREAEEA,CCCHHHHHHHHHHHHHHHTC,CCCHHHHHHHHHHHHHHHCC,20,False,XRAY,1.45,0.19,0.22
3,1QOW,D,CTFTLPGGGGVCTLTSECI*,CCTTSCTTCSSTTSSTTCCC,CCCCCCCCCCCCCCCCCCCC,20,True,XRAY,1.06,0.14,1.00
4,1RDQ,I,TTYADFIASGRTGRRNAIHD,CHHHHHHTSSCSSCCCCEEC,CHHHHHHCCCCCCCCCCEEC,20,False,XRAY,1.26,0.13,0.16
...,...,...,...,...,...,...,...,...,...,...,...
9073,4LGY,A,GDGLVPRGSHMMEILRGSPALSAFRINKLLARFQAANLQVHNIYAE...,CCEEEEETTEEEEEEEEEECCCHHHHHHHHHHHHHTTCCCCEEEEE...,CCEEEEECCEEEEEEEEEECCCHHHHHHHHHHHHHCCCCCCEEEEE...,1305,False,XRAY,1.48,0.14,0.17
9074,5XH6,A,GSHMTQFEGFTNLYQVSKTLRFELIPQGKTLKHIQEQGFIEEDKAR...,CCCCCCGGGCBSSSCEEEEEEEEEEECTTHHHHHHHHTHHHHHHHH...,CCCCCCHHHCECCCCEEEEEEEEEEECCCHHHHHHHHCHHHHHHHH...,1310,False,XRAY,2.00,0.18,0.21
9075,5B2R,B,GSGHMDKKYSIGLAIGTNSVGWAVITDEYKVPSKKFKVLGNTDRHS...,CCCCCCCCCEEEEEECSSEEEEEEECTTSCCCEEEEEEEESSSCCE...,CCCCCCCCCEEEEEECCCEEEEEEECCCCCCCEEEEEEEECCCCCE...,1372,False,XRAY,2.00,0.20,0.23
9076,5WLH,A,SNAMKISKVREENRGAKLTVNAKTAVVSENRSQEGILYNDPSRYGK...,CCCCCCCCCCCCCCCCCTTSSEEEEEEESSSCEEEEEEESSCCTTT...,CCCCCCCCCCCCCCCCCCCCCEEEEEEECCCCEEEEEEECCCCCCC...,1440,False,XRAY,1.80,0.19,0.22


In [5]:
df[1:5]

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa,Exptl.,resolution,R-factor,FreeRvalue
1,1LM8,H,DLDLEMLAPYIPMDDDFQLR,CCCCCCCCCBCCSCCCEECC,CCCCCCCCCECCCCCCEECC,20,False,XRAY,1.85,0.20,0.24
2,1O06,A,EEDPDLKAAIQESLREAEEA,CCCHHHHHHHHHHHHHHHTC,CCCHHHHHHHHHHHHHHHCC,20,False,XRAY,1.45,0.19,0.22
3,1QOW,D,CTFTLPGGGGVCTLTSECI*,CCTTSCTTCSSTTSSTTCCC,CCCCCCCCCCCCCCCCCCCC,20,True,XRAY,1.06,0.14,1.00
4,1RDQ,I,TTYADFIASGRTGRRNAIHD,CHHHHHHTSSCSSCCCCEEC,CHHHHHHCCCCCCCCCCEEC,20,False,XRAY,1.26,0.13,0.16


In [15]:
df[-1:]["sst8"].unique()

array(['CCCCCEEECCEEEEECSSEEEEEEECEETTEEGGGCCCEEEEEEEECTTTSCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHTSCCCCCHHHHHHHHHHHSCCCBCSCCCCCCCCCCCCCCCCCHHHHHHHHHHCTTCCCCCHHHHHHTTSHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCCHHHHHHTHHHHGGGHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHTCHHHHHHHHHHHTTSCCGGGCSCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHCCCBHHHHHHHHHHHHHCSCCSSHHHHHHHHHHHTTTTTTCCHHHHHHHHHHHHTCCHHHHHHHHCSGGGSSSCCCCHHHHHHHHHHHHHHTCCCCTTCGGGSTTSTTCHHHHHHHHHHHTTTSCHHHHHTTSCGGGGSCCCCCCCSSSCCCCCBEEECHHHHHHHCTTHHHHHHHHHTSHHHHHHHTTHHHHHHHCBCTTSSBSEESSCCSCHHHHTTCEEHHHHHHHHHHHHHTBCGGGCTTCHHHHHHHHHHHHCCCCCCCCCCCCSHHHHHHHHHCCSCGGGCCCEETTEECTTSHHHHHHHHHHHHHHHHTTCCCCCEEEEEETTTTEEEEEEEECCTTCSEEECCCCCCBGGGCHHHHHHHHHTSCHHHHHHHHTCCCHHHHHHHHHHHSTTHHHHHHHHHHHHHHSSSCHHHHHHHHHHHTTSSSHHHHHHHHHHHCCCCCCCCCCCCHHHHHHHHHHCCCCTTTHHHHHHHHTTSCSHHHHHHHHHHHSSCCCSBCSSCHHHHHHHHHHTCEECCCCCCCCCCCCCCCEESBCCCCBCCCCCCCHHHHHHHHHHHHHHHHHTHHHHHHHHHTTCEEEEEEEEEECHHHHHHHHHHHTTCCCCHHHHHHHHTTSTTTTTHHHHHHHHHTCCCCCCCCCCCCCCCCCCCCCCEEEESSCCCCCCCCCCCSGGGEEE

In [17]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

amino_acids = list(d.values())

amino_to_int_dict = {i:idx+3 for idx,i in enumerate(amino_acids)}
amino_to_int_dict["<PAD>"] = 0
amino_to_int_dict["<MASK>"] = 1
amino_to_int_dict["<END>"] = 0
int_to_amino_dict = {v:k for v,k in amino_to_int_dict.items()}


sst2_to_int_dict = {"<PAD>":0,"<END>":0,"H":1,"C":2,"E":2}
sst3_to_int_dict = {"<PAD>":0,"<END>":0,"H":1,"C":2,"E":3}
sst8_to_int_dict = {"<PAD>":0,"<END>":0,"H":1,"C":2,"E":3,"B":4,"G":5,"I":6,"T":7,"S":8}

int_to_sst2 = {v:k for v,k in sst2_to_int_dict.items()}
int_to_sst3 = {v:k for v,k in sst3_to_int_dict.items()}
int_to_sst8 = {v:k for v,k in sst8_to_int_dict.items()}

In [18]:
def tokenize_from_dict(sequence,tokenizer_dict,length=None):

	if length is None:
		length = len(sequence)

	seq = []

	for i in range(length):

		if i>len(sequence):
			seq.append(tokenizer_dict["<PAD>"])
		elif i == len(sequence):
			seq.append(tokenizer_dict["<END>"])
		else:
			seq.append(tokenizer_dict[sequence[i]])

	return torch.tensor(seq)


In [19]:
df[1:5]

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa,Exptl.,resolution,R-factor,FreeRvalue
1,1LM8,H,DLDLEMLAPYIPMDDDFQLR,CCCCCCCCCBCCSCCCEECC,CCCCCCCCCECCCCCCEECC,20,False,XRAY,1.85,0.20,0.24
2,1O06,A,EEDPDLKAAIQESLREAEEA,CCCHHHHHHHHHHHHHHHTC,CCCHHHHHHHHHHHHHHHCC,20,False,XRAY,1.45,0.19,0.22
3,1QOW,D,CTFTLPGGGGVCTLTSECI*,CCTTSCTTCSSTTSSTTCCC,CCCCCCCCCCCCCCCCCCCC,20,True,XRAY,1.06,0.14,1.00
4,1RDQ,I,TTYADFIASGRTGRRNAIHD,CHHHHHHTSSCSSCCCCEEC,CHHHHHHCCCCCCCCCCEEC,20,False,XRAY,1.26,0.13,0.16


In [34]:
def get_xy(df,l_max):

	inputs = []
	sst2_labels = []
	sst3_labels = []
	sst8_labels = []

	for index,row in df.iterrows():
		# we ignore sequences with special amino acids
		if "*" in row["seq"]:
			continue

		if len(row["seq"]) <= l_max-1:
			inputs.append(tokenize_from_dict(row["seq"],amino_to_int_dict,l_max))
			sst2_labels.append(tokenize_from_dict(row["sst3"],sst2_to_int_dict,l_max))
			sst3_labels.append(tokenize_from_dict(row["sst3"],sst3_to_int_dict,l_max))
			sst8_labels.append(tokenize_from_dict(row["sst8"],sst8_to_int_dict,l_max))

	return torch.stack(inputs),torch.stack(sst2_labels),torch.stack(sst3_labels),torch.stack(sst8_labels)

In [35]:
inputs,sst2_labels,sst3_labels,sst8_labels = get_xy(df,384)

In [37]:
sst3_labels.shape

torch.Size([7578, 384])